<a href="https://colab.research.google.com/github/vishalsalgond/Jupyter_Notebooks/blob/master/imdb_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
print(tf.__version__)

2.2.0


# Import Dataset

In [0]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete8E5SB8/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete8E5SB8/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete8E5SB8/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## Preprocessing

In [0]:
import numpy as np
train_data, test_data = imdb['train'], imdb['test']

In [0]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

In [0]:
for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())

for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

In [0]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

## Initializing Hyperparameters

In [0]:
vocab_size = 10000
embedding_dim = 16
max_len = 120
trunc_type = "post"
oov_tok = "<OOV>"

## Tokenization and padding

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

In [0]:
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen = max_len, truncating=trunc_type)

In [0]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_len)

## Defining the Model

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [0]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


# Training the model

In [0]:
num_epochs = 10
model.fit(padded,
          training_labels_final,
          epochs=num_epochs,
          validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.4913 - accuracy: 0.7444 - val_loss: 0.3559 - val_accuracy: 0.8429
Epoch 2/10
782/782 [==============================] - 4s 4ms/step - loss: 0.2314 - accuracy: 0.9108 - val_loss: 0.3763 - val_accuracy: 0.8364
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0787 - accuracy: 0.9814 - val_loss: 0.4609 - val_accuracy: 0.8244
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0186 - accuracy: 0.9981 - val_loss: 0.5378 - val_accuracy: 0.8220
Epoch 5/10
782/782 [==============================] - 4s 4ms/step - loss: 0.0053 - accuracy: 0.9996 - val_loss: 0.5935 - val_accuracy: 0.8226
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.6434 - val_accuracy: 0.8234
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 7.9033e-04 - accuracy: 1.0000 - val_loss: 0.6836 - val_accuracy: 0.8244
Ep

In [0]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


## Reverse the dictionary

In [0]:
reverse_word_index = dict([(value, key) for (key,value) in word_index.items()])

## Download files to visualize

In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:
try:
    from google.colab import files
except IMportError:
    pass
else:
    files.download('vecs.tsv')
    files.download('meta.tsv')

In [0]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 68, 105, 12, 7, 491, 1216]]
